In [1]:
# Importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from ipywidgets import interactive, FloatSlider

# Initial values
E0 = 1.0     # Initial concentration of E
S0 = 110.0     # Initial concentration of S
k_m0 = 55   # Example initial value for k_m
k20 = 96    # Example initial value for k2

# Time points
t_dense = np.linspace(0, 0.02, 200)  # Densely sampled time points for 0.00 to 0.02
t_rest = np.linspace(0.02, 20, 400)  # Remaining time points
t = np.concatenate((t_dense, t_rest[1:]))  # Combine and avoid duplicate at 0.02

# Define the differential equations for the enzymatic reaction
def reaction_system(y, t, k_m, k2):
    E, S, ES, P = y

    # Calculate k1 and k_1 from k_m and k2
    # Assuming k_m = (k_1 + k2) / k1, we need to choose a value for k1 or k_1
    # For example, let's assume k1 = 1 for calculation purpose
    k1 = 1.0
    k_1 = k_m * k1 - k2

    dE_dt = -k1 * E * S + (k_1 + k2) * ES  # Rate of change of E
    dS_dt = -k1 * E * S + k_1 * ES         # Rate of change of S
    dES_dt = k1 * E * S - (k_1 + k2) * ES  # Rate of change of ES
    dP_dt = k2 * ES                        # Rate of change of P
    return [dE_dt, dS_dt, dES_dt, dP_dt]

def compute_sliding_window_variance(data, window_size):
    variance = np.array([np.var(data[i:i+window_size]) for i in range(len(data)-window_size)])
    return variance

def plot_enzyme_kinetics(initial_S, initial_E, k_m, k2):
    
    # Update initial concentrations with slider values
    y0 = [initial_E, initial_S, 0.0, 0.0]

    # Solving the system of differential equations
    solution = odeint(reaction_system, y0, t, args=(k_m, k2))
    E, S, ES, P = solution.T
                
    plt.figure(figsize=(10, 6))
    plt.plot(t, E, label='[E]')
    plt.plot(t, S, label='[S]')
    plt.plot(t, ES, label='[ES]')
    plt.plot(t, P, label='[P]')
    plt.yscale('asinh')
    plt.ylim(bottom=0, top=2000)
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.title('Enzyme Kinetics')
    plt.legend(loc='center right')
    plt.grid(True)

    # Calculate the first derivative (rate of change) of ES
    dES_dt = np.gradient(solution[:, 2], t)

    # Compute the sliding window variance
    #window_size = 40  # Adjust the window size as needed
    #variance = compute_sliding_window_variance(dES_dt, window_size)

    # Determine the threshold for variance
    #variance_threshold = 6e-10  # Adjust as needed
    #sustained_period = 20  # Period for which variance should be below threshold

    # Find the index where the variance is below the threshold for a sustained period
    #steady_state_index = -1
    #for i in range(len(variance) - sustained_period):
    #    if all(variance[i:i+sustained_period] < variance_threshold):
    #        steady_state_index = i + window_size // 2
    #        break
    
    # Check if steady_state_index was updated
    #if steady_state_index != -1:
    #    steady_state_start = t[steady_state_index]

    #    # Shading the steady state region
    #    plt.axvspan(steady_state_start, max(t), color='grey', alpha=0.1)

    #    # Choose y-coordinate for the bracket and label, below the x-axis
    #    bracket_y_position = -0.05 * plt.ylim()[1]  # Example: 5% below the lower y-limit
    #    label_y_position = -0.10 * plt.ylim()[1]   # Example: 10% below the lower y-limit

    #    # Adding a bracket to indicate the extent of the steady state region
    #    plt.annotate('', xy=(steady_state_start, bracket_y_position), xytext=(max(t), bracket_y_position), 
    #                 xycoords='data', textcoords='data',
    #                 arrowprops=dict(arrowstyle='|-|', lw=1))

    # Adding a label below the x-axis
    #plt.text((steady_state_start + max(t))/2, label_y_position, 'Steady state', 
    #         horizontalalignment='center', verticalalignment='center', fontsize=12)
        
    # Creating an inset
    # Parameters are [x-coordinate, y-coordinate, width, height] of the inset
    ax_inset = inset_axes(plt.gca(), width="20%", height="20%", loc=1) # loc is the location
    ax_inset.plot(t, E, label='[E]')
    ax_inset.plot(t, S, label='[S]')
    ax_inset.plot(t, ES, label='[ES]')
    ax_inset.plot(t, P, label='[P]')
    # Customize the range and labels of the inset as needed
    # For example, zooming in on the time range from 0 to 10
    ax_inset.set_xlim(0, 0.02)
    ax_inset.set_ylim(0, 1)
    ax_inset.set_title("Initial velocity")
    ax_inset.grid(True)
    
    # Calculate the initial slope (initial velocity)
    delta_t = t[100] - t[0]  # Time interval between points
    initial_slope = (P[100] - P[0]) / delta_t  # Slope (change in P over change in time)

    # Extend the range for drawing the tangent
    tangent_start_time = 0  # Starting time for the tangent line
    tangent_end_time = 10   # Extend this as needed to make the tangent line longer
    tangent_time = np.linspace(tangent_start_time, tangent_end_time, 100)

    # Calculate the tangent line over the extended range
    tangent_line = P[0] + initial_slope * (tangent_time - t[0])
    
    # Draw the tangent line in the inset
    ax_inset.plot(tangent_time, tangent_line, 'r--')

    # Bottom right corner coordinates (You might need to adjust these values)
    x_pos = 0.95 * ax_inset.get_xlim()[1]  # 95% of the way to the right
    y_pos = 0.05 * ax_inset.get_ylim()[1]  # 5% of the way up from the bottom

    # Place the label in the bottom right corner
    ax_inset.text(x_pos, y_pos, '$v_0$', horizontalalignment='right', verticalalignment='bottom', fontsize=12, color='red')
    
    plt.show()
    
# Interactive sliders for k_m and k2
interactive_plot = interactive(plot_enzyme_kinetics, 
                               initial_S=FloatSlider(value=S0, min=0.0, max=2000, step=10, description='$[S_0]$'),
                               initial_E=FloatSlider(value=E0, min=0.0, max=10.0, step=1, description='$[E_T]$'),
                               k_m=FloatSlider(value=k_m0, min=0.0, max=100, step=1, description='$[k_M]$'),
                               k2=FloatSlider(value=k20, min=0.0, max=100, step=1, description='$[k_{{cat}}]$'))

# Display the interactive plot with sliders
interactive_plot


interactive(children=(FloatSlider(value=110.0, description='$[S_0]$', max=2000.0, step=10.0), FloatSlider(valu…